# Maleri Classifier v2

Setup

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *

In [3]:
types = 'Abildgaard, Nicolai','Dahl, J.C','Eckersberg, C.W','Hammershøi, Vilhelm','Hansen, Constantin','Jensen, C.A','Juel, Jens','Krøyer, P.S','Kyhn, Vilhelm','Købke, Christen','Lorentzen, C.A','Marstrand, Wilhelm','Pauelsen, Erik','Philipsen, Theodor','Skovgaard, P.C'
path = Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier')

File tests

In [4]:
fns = get_image_files(path)
fns

(#604) [Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/010.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/002.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/003.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/006.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/008.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/001.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/009.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/007.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/004.jpg'),Path('gdrive/MyDrive/AI Machine Learning/images/Maleri Classifier/Dahl, J.C/005.jpg')...]

In [5]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [6]:
dls = data.dataloaders(path)

Model training

In [7]:
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = data.dataloaders(path)

In [8]:
learn = cnn_learner(dls, resnet152, metrics=error_rate)
learn.fine_tune(4)

epoch,train_loss,valid_loss,error_rate,time


KeyboardInterrupt: ignored

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Exporting

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('Images/painting.jpg')

('Marstrand, Wilhelm',
 tensor(11),
 tensor([0.2217, 0.0525, 0.0057, 0.0223, 0.0413, 0.0789, 0.0022, 0.0418, 0.0257, 0.0107, 0.1611, 0.2983, 0.0207, 0.0045, 0.0125]))

In [ ]:
learn_inf.dls.vocab

['Abildgaard, Nicolai', 'Dahl, J.C', 'Eckersberg, C.W', 'Hammershoi, Vilhelm', 'Hansen, Constantin', 'Jensen, C.A', 'Juel, Jens', 'Kobke, Christen', 'Kroyer, P.S', 'Kyhn, Vilhelm', 'Lorentzen, C.A', 'Marstrand, Wilhelm', 'Pauelsen, Erik', 'Philipsen, Theodor', 'Skovgaard, P.C']

App

In [ ]:
learn_inf = load_learner('export.pkl')

In [ ]:
img = PILImage.create('Images/painting.jpg')

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: Marstrand, Wilhelm; Probability: 0.2983')

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your painting!'), 
      btn_upload, btn_run, out_pl, lbl_pred])